# Isolated Farming Idea Coputations and References

### Short Description

Vertical farming is becaming a fancy word and it seems very successfull, one can obtain very large (up to a factor 50 ro more) increase in efficiency of production of almost any vegetables. This has to do in mostly with enhanced efficiency of photosintesis [[1](https://www.nature.com/articles/ncomms12699)] that can be achieved in controlled conditions. See next cell for details on the calculation of the factor 50.




[1] - https://www.nature.com/articles/ncomms12699

[2] - https://www.pnas.org/content/117/32/19131

[3] - https://data.worldbank.org/indicator/AG.YLD.CREL.KG

[4] - https://www.agriculture.gov.au/abares/research-topics/agricultural-outlook/wheat

[5] - https://en.wikipedia.org/wiki/Solar_irradiance#Irradiance_on_Earth's_surface


In [19]:
# imports
import math

In [20]:
# Computation of efficiency boost of indor farming, from [1,2] above.
wheater_conditions = 0.35
light_reflection = 0.5
sun_light_half_time = 0.5
# other_condition = 0.1  # this is up to 0.1, and is because of temperature, etc...
green_light_not_absorbed = 0.43 # will not consider this 
more_harvest = 5 # from [2]

boost_factor_est = more_harvest /(wheater_conditions * light_reflection * sun_light_half_time )
boost_factor_algae = more_harvest /(wheater_conditions * light_reflection * sun_light_half_time ) #FIXME

wheat_per_ha_outdoor = 3.2 # this is tonne/ha according to [2], but [3] says that is 4 t/ha (we can maybe assume better yeald indoor as well)
wheat_per_ha_indoor = 194 # tonne/ha [2]
boost_factor_wheat = wheat_per_ha_indoor / wheat_per_ha_outdoor


print("Estimated: Outdoor = Indoor / ", int(boost_factor_est))
print("Algae: Outdoor = Indoor / ", int(boost_factor_algae)) # FIXME
print("Wheat: Outdoor = Indoor / ", int(boost_factor_wheat))


Estimated: Outdoor = Indoor /  57
Algae: Outdoor = Indoor /  57
Wheat: Outdoor = Indoor /  60


In [21]:
# Parameters
meter = 1
hectare = 10000*meter**2
acre  = 0.404686 * hectare

indoor_ha_equivalent = hectare / boost_factor_wheat
print("1 hectare outoor = Indoor m^2 ", int(indoor_ha_equivalent) )

1 hectare outoor = Indoor m^2  164


In [34]:
pod_od_inner  =  1 # m
pod_lenght = 5 # m
pod_area =  pod_od_inner * math.pi * pod_lenght * meter**2
pod_ha_equivalent = indoor_ha_equivalent / pod_area

print("Number of pods equivalent to an hectar (for wheat)", pod_ha_equivalent)

Number of pods equivalent to an hectar (for wheat) 10.500944698846704


In [23]:
# revenue per hectar wheat
price_per_tonne_wheat = 245 # USD/tonne
revenue_wheat_outdoor = price_per_tonne_wheat * wheat_per_ha_outdoor 
revenue_per_pod_year = revenue_wheat_outdoor / pod_ha_equivalent
print("revenue USD 1ha year wheat ", revenue_wheat_outdoor)
print(" revenue USD per pod year wheat", revenue_per_pod_year )

revenue USD 1ha year wheat  784.0
 revenue USD per pod year wheat 19.012


In [24]:
pod_cost = 500
pod_costs_operation = 0.5
year_to_repay = pod_cost / pod_costs_operation / revenue_per_pod_year
print("year needed to repay the pod cost", year_to_repay)
print("kg/pod/year/harvest", wheat_per_ha_indoor * 1 * pod_area / hectare * 1000/more_harvest)
print("cross check ", wheat_per_ha_outdoor / pod_ha_equivalent * 1000 / more_harvest)

year needed to repay the pod cost 52.598358931201346
kg/pod/year/harvest 15.520000000000001
cross check  15.520000000000001


# TODO
- See soy boost and algae boost factors
- see cost of electricity for outputting amount of light in [2]

In [25]:
# cost of electricity for wheat
micro = 1E-6
nano= 1E-9
avogadro = 6.02214076E23


def photon_Energy_joule(lambda_nm):
    plank = 6.62607015E-34 # Js
    c = 299792458 # m/s
    E = c * plank / (lambda_nm * nano)
    return E

def watt_from_Lambda_and_intensity(lambda_nm, intensity_micro_mole):
    E_of_photon = photon_Energy_joule(lambda_nm)
    return E_of_photon * avogadro * intensity_micro_mole * micro
    


In [26]:
light_watt_needed = watt_from_Lambda_and_intensity(500,2000) # from best on [2]
LED_max_efficiency = 0.66 # W/W from [1]
LED_low_efficiency = 0.25 # W/W from [1]

electrical_power_per_m2 = light_watt_needed / LED_max_efficiency
print("photon E [W/m^2]", int(light_watt_needed))
print("electrical power [W]/ m^2 ",int(electrical_power_per_m2) ) 
# in agreement with [5] 1000 W/m^2 sun average considering factor 0.5 due to wavelenght absorption

photon E [W/m^2] 478
electrical power [W]/ m^2  725


In [27]:
power_per_pod = electrical_power_per_m2 * pod_area
energy_price = 0.1 # Eur per KWh
print("power per pod [W]", int(power_per_pod))
print("cost per year Eur/pod", int(energy_price * power_per_pod/1000 * 22 *365 ))
print("Cost / Revenue", energy_price * power_per_pod/1000 * 24 *365 / revenue_per_pod_year)

power per pod [W] 2900
cost per year Eur/pod 2328
Cost / Revenue 133.6226198439245


# Soybean

Case of soybean is different because is a short-day plant [6].

In [8] it says:
Since lighting schemes with more than 12-h light are not suitable for short-day crops, but as enhanced photosynthesis rates may shorten cycle durations, we evaluated the impact of light intensity on flowering time and plant morphology for a 10-h lighting scheme. Increasing light intensity above 1000 µmol/m2 s led to a ~ 2 days earlier flowering (21.8 ± 1.1 vs. 23.9 ± 1.1 for light intensity of 500–900 µmol/m2 s) and shorter plant stature. These findings point to a further speed-up option by increasing light intensity, but these subtle changes will come with additional expenses due to higher energy costs and more expensive LED systems. Thus, for a well operating system a light intensity of ~ 500 µmol/m2 s at 50 cm from the light source should suffice to achieve fast generation times on a moderate budget.


[6] - https://academic.oup.com/pcp/article/60/1/77/5096780

[7] - https://www.frontiersin.org/articles/10.3389/fpls.2018.01952/full

[8] - https://link.springer.com/article/10.1007/s00122-020-03601-4



In [28]:
hours_light_on_day_soy = 14 #h
light_intensity_soy = 500 # mu mol/m^2/s  best according to [7-8] 

light_watt_needed_soy = watt_from_Lambda_and_intensity(500, light_intensity_soy)
electrical_power_per_m2_soy = light_watt_needed_soy / LED_max_efficiency
print("photon E [W/m^2]", int(light_watt_needed_soy))
print("electrical power [W]/ m^2 ",int(electrical_power_per_m2_soy) ) 

photon E [W/m^2] 119
electrical power [W]/ m^2  181


In [29]:
# assumes say ratio as for wheat - reasonable because of [6]
power_per_pod_soy = electrical_power_per_m2_soy * pod_area
energy_price = 0.1 # Eur per KWh
print("power per pod [W]", int(power_per_pod_soy))
print("cost per year Eur/pod", int(energy_price * power_per_pod_soy/1000 * hours_light_on_day_soy *365 ))
#print("Cost / Revenue", energy_price * power_per_pod_/1000 * 24 *365 / revenue_per_pod_year)

power per pod [W] 725
cost per year Eur/pod 370


In [30]:
average_yeald_soy_hectare = 3.78 # tonne
price_soy_tonne =  379 # USD
revenue_per_pod_soy = price_soy_tonne * average_yeald_soy_hectare / pod_ha_equivalent

print("Revenue soy per pod/year USD", int(revenue_per_pod_soy))

Revenue soy per pod/year USD 34


# Construction costs

In [39]:
ss_density = 7.8 # tonne / m^3
ss_lenght = 5 # m
ss_OD_inner = 1 # m
ss_OD_outer = 1.03 # m  (3cm spacing)
ss_thick = 1E-3 # m (0.5mm)


def tube_volume(l,od,t):
    return math.pi * l * t *(od - t)
    
ss_tube_mass_inner = ss_density * tube_volume(ss_lenght, ss_OD_inner, ss_thick)
ss_tube_mass_outer = ss_density * tube_volume(ss_lenght, ss_OD_outer, ss_thick)

print("mass in kg ", ss_tube_mass_inner * 1000)

mass in kg  122.39959137651195


In [40]:
ss_tube_cost_tonne = 1200 # $ per tonne

chamber_cost = ss_tube_cost_tonne * ( ss_tube_mass_inner + ss_tube_mass_outer)
flange_cost_prediction = 100 # $ + hope 

pod_structural_cost = chamber_cost + 2 * flange_cost_prediction

print("price per tube $", int(ss_tube_cost_tonne * ss_tube_mass_outer), "chamber cost $", int(chamber_cost))
print("pod raw cost $", int(pod_structural_cost) )

price per tube $ 151 chamber cost $ 298
pod raw cost $ 498
